# Han Jeon
## ML - Final Project
### Predicting Yelp Reviews based off of commentary.

In [96]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import sqlite3

In [15]:
os.getcwd()

'/Users/heoun/Google Drive/School/Machine Learning & Predictive Analysis - MSCA 31009/Assignments'

In [16]:
os.chdir("/Users/heoun/Desktop/yelp-dataset")

In [85]:
os.getcwd()

'/Users/heoun/Desktop/yelp-dataset'

# EDA

In [23]:
df_bus = pd.read_json('yelp_academic_dataset_business.json', lines = True)
df_bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [90]:
df_bus.describe()

,latitude,longitude,stars,review_count,is_open
count,192609.000000,192609.000000,192609.000000,192609.000000,192609.000000
mean,38.541803,-97.594785,3.585627,33.538962,0.823040
std,4.941964,16.697725,1.018458,110.135224,0.381635
min,33.204642,-115.493471,1.000000,3.000000,0.000000
25%,33.637408,-112.274677,3.000000,4.000000,1.000000
50%,36.144815,-111.759323,3.500000,9.000000,1.000000
75%,43.602989,-79.983614,4.500000,25.000000,1.000000
max,51.299943,-72.911982,5.000000,8348.000000,1.000000


In [53]:
print(df_bus['city'].nunique())
print('There is 1204 unique cities in this dataset')

1204
There is 1204 unique cities in this dataset


In [72]:
df_bus.groupby(['state','city']).size().sort_values(ascending=False).head(10)

state  city      
NV     Las Vegas     29361
ON     Toronto       18904
AZ     Phoenix       18764
NC     Charlotte      9507
AZ     Scottsdale     8837
AB     Calgary        7735
PA     Pittsburgh     7016
QC     Montréal       6449
AZ     Mesa           6080
NV     Henderson      4892
dtype: int64

Out of the 1204 unique cities; there's about 5 cities we could potentially focus on to give us the most amount of information.

In [71]:
df_bus.groupby(['state']).size().sort_values(ascending=False).head(10)

state
AZ    56686
NV    36312
ON    33412
NC    14720
OH    14697
PA    11216
QC     9219
AB     8012
WI     5154
IL     1932
dtype: int64

It may be better to foucs on the state of Arizona or Nevada as there may be more data to work with.

In [76]:
df_bus.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      28836
categories        482
hours           44830
dtype: int64

In [24]:
df_reviews = pd.read_json('yelp_academic_dataset_review.json', lines = True)
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [94]:
df_reviews[['stars','useful','funny','cool','date']].describe()

,stars,useful,funny,cool
count,6.685900e+06,6.685900e+06,6.685900e+06,6.685900e+06
mean,3.716199e+00,1.354134e+00,4.827667e-01,5.787708e-01
std,1.463643e+00,3.700192e+00,2.378646e+00,2.359024e+00
min,1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00
max,5.000000e+00,1.241000e+03,1.290000e+03,5.060000e+02


In [74]:
df_reviews.groupby(['stars']).size().sort_values(ascending=False).head(10)

stars
5    2933082
4    1468985
1    1002159
3     739280
2     542394
dtype: int64

In [79]:
df_reviews.groupby(['useful']).size().sort_values(ascending=False).head(10)

useful
0    3570453
1    1430632
2     669040
3     349808
4     201772
5     125187
6      82613
7      56609
8      41111
9      30155
dtype: int64

In [81]:
df_reviews.groupby(['funny']).size().sort_values(ascending=False).head(10)

funny
0    5312173
1     813583
2     255974
3     111881
4      59525
5      35781
6      23065
7      15972
8      11461
9       8387
dtype: int64

In [82]:
df_reviews.groupby(['cool']).size().sort_values(ascending=False).head(10)

cool
0    5013287
1    1013484
2     313433
3     129433
4      65993
5      38944
6      25562
7      17571
8      12528
9       9426
dtype: int64

In [86]:
df_reviews.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [25]:
df_tip = pd.read_json('yelp_academic_dataset_tip.json', lines = True)
df_tip.head()

,user_id,business_id,text,date,compliment_count
0,UPw5DWs_b-e2JRBS-t37Ag,VaKXUpmWTTWDKbpJ3aQdMw,"Great for watching games, ufc, and whatever el...",2014-03-27 03:51:24,0
1,Ocha4kZBHb4JK0lOWvE0sg,OPiPeoJiv92rENwbq76orA,Happy Hour 2-4 daily with 1/2 price drinks and...,2013-05-25 06:00:56,0
2,jRyO2V1pA4CdVVqCIOPc1Q,5KheTjYPu1HcQzQFtm4_vw,Good chips and salsa. Loud at times. Good serv...,2011-12-26 01:46:17,0
3,FuTJWFYm4UKqewaosss1KA,TkoyGi8J7YFjA6SbaRzrxg,The setting and decoration here is amazing. Co...,2014-03-23 21:32:49,0
4,LUlKtaM3nXd-E4N4uOk_fQ,AkL6Ous6A1atZejfZXn1Bg,Molly is definately taking a picture with Sant...,2012-10-06 00:19:27,0


In [92]:
df_tip.describe()

,compliment_count
count,1.223094e+06
mean,1.647380e-02
std,1.456401e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.500000e+01


In [88]:
df_tip.isnull().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [89]:
df_tip.groupby(['compliment_count']).size().sort_values(ascending=False).head(10)

compliment_count
0     1205140
1       16239
2        1402
3         227
4          51
5          18
6           8
7           4
15          1
12          1
dtype: int64

In [144]:
df_merged = df_reviews.merge(df_bus,how = 'left', left_on= "business_id", right_on = 'business_id')

In [145]:
df_merged.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,MountainView Hospital,...,NV,89128,36.215546,-115.248798,2.5,238,1,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Fitness & Instruction, Doctors, Health & Medic...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,Kelly Cardenas Salon - Hard Rock Hotel & Casino,...,NV,89169,36.110083,-115.153871,4.5,66,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Beauty & Spas, Hair Stylists, Makeup Artists, ...","{'Monday': '10:0-17:0', 'Tuesday': '10:0-19:0'..."
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,J. Philipp Centers for Family and Cosmetic Den...,...,AZ,85286,33.259702,-111.790203,3.5,39,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Health & Medical, Cosmetic Dentists, Orthodont...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-17:0', '..."
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,Regal Beagle Pub,...,AB,T2J 3G8,50.969721,-114.070115,3.0,13,1,"{'HasTV': 'True', 'RestaurantsReservations': '...","Bars, Pubs, Nightlife, Tapas Bars, Restaurants",None
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,White Teeth AZ,...,AZ,85251,33.496713,-111.927958,4.5,75,1,"{'WheelchairAccessible': 'False', 'BusinessAcc...","Teeth Whitening, Beauty & Spas","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."


In [101]:
df_merged.groupby('city').size().sort_values(ascending = False).head(10)

city
Las Vegas     2030798
Phoenix        734136
Toronto        525437
Scottsdale     387550
Charlotte      309425
Pittsburgh     225496
Henderson      219880
Tempe          202074
Mesa           168311
Chandler       157581
dtype: int64

In [116]:
df_merged[df_merged['city'] == 'Charlotte'].groupby(['city','business_id']).size().sort_values(ascending = False).head(10)

city       business_id           
Charlotte  yQab5dxZzgBLTEHCw9V7_w    1895
           RAh9WCQAuocM7hYM5_6tnw    1645
           gG9z6zr_49LocyCTvSFg0w    1598
           WbJ1LRQdOuYYlRLyTkuuxw    1561
           RVQE2Z2uky4c0-njFQO66g    1525
           T2tEMLpTeSMxLKpxwFdS3g    1129
           01fuY2NNscttoTxOYbuZXw    1024
           ALwAlxItASeEs2vYAeLXHA     967
           NFm869_w6cvVaWaNpAzjeA     962
           aO1gAp41n8w8zpxTiHdLNA     907
dtype: int64

In [143]:
df_merged[['city','review_id']].groupby('city').size().sort_values(ascending = False)

city
Las Vegas     2030798
Phoenix        734136
Toronto        525437
Scottsdale     387550
Charlotte      309425
               ...   
Suprise             3
Perry Twp           3
Henryville          3
CARSON              3
                    3
Length: 1204, dtype: int64

In [146]:
df_char = df_merged[df_merged['city'] == 'Charlotte']
df_char.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
50,-QxhbVQrcnVWGiua9r44JA,jsOz-dWToun2VsFcFAfGww,FBSWwaE6gR7KAOyG1QhakQ,5,0,0,0,Such a unique place & a great find. Our waitre...,2015-01-03 21:11:31,Block & Grinder,...,NC,28211,35.170504,-80.806674,4.0,223,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Butcher, Nightlife, Food, American (New), Rest...","{'Tuesday': '11:30-22:0', 'Wednesday': '11:30-..."
52,Saae_VVf_v0fyNmB6Dc4kw,PttoI2Meg76KVB8JxjC5Xw,rBo_suDMdGmFQ0TtYg3b2Q,1,4,0,0,I've never experienced discrimination as I did...,2014-07-13 16:17:18,Oak Room,...,NC,28203,35.216296,-80.854933,2.0,33,1,"{'OutdoorSeating': 'True', 'RestaurantsGoodFor...","Bars, Nightlife, Cocktail Bars, Dance Clubs","{'Wednesday': '22:0-2:0', 'Saturday': '22:0-2:0'}"
61,HYBzY7zN_YBHtQu6qkD3Ng,OnYEKooAtI2zTRqBLLIfvA,FBSWwaE6gR7KAOyG1QhakQ,5,0,0,0,Easily entering my top 3 restaurants in Charlo...,2016-03-07 00:20:06,Block & Grinder,...,NC,28211,35.170504,-80.806674,4.0,223,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Butcher, Nightlife, Food, American (New), Rest...","{'Tuesday': '11:30-22:0', 'Wednesday': '11:30-..."
64,ASZha3MiQJghtt3n5OGIsA,Fyn9aNWs0Qcz6T-uIJal4w,ZCsvwa_dsqNB8d8tYHPdjw,4,0,0,0,This random hole in the wall is located in the...,2012-04-28 13:35:52,Sir Edmond Halley's,...,NC,28209,35.175061,-80.848894,3.5,156,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Pubs, Bars, Fish & Chips, Nightlife","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ..."
76,LrO2MkIWIjAoOeS5WImnrQ,hzny0aF2jcUaY8rXN2bfJQ,CGUK3cd2gxp2q3KPY19Oog,5,0,0,0,Yum!! Had the kale grits and would definitely ...,2018-04-15 11:34:34,Roots Cafe,...,NC,28203,35.206507,-80.860258,4.0,104,1,"{'BusinessParking': '{'garage': False, 'street...","Breakfast & Brunch, Restaurants, Coffee & Tea,...","{'Tuesday': '7:0-21:0', 'Wednesday': '7:0-21:0..."


In [147]:
df_char.to_csv('df_char.csv')